# 07-router-pattern.ipynb

- 멀티 에이전트 패턴
- Supervisor (관리자 Agent) 없음
- 시작과 동시에 어떤 어떤 agent를 사용해야 할 지 판단 (1_n개)
- n개의 agent를 병렬 실행
- 각 agent들이 만든 결과를 종합

길을 뚫어주는 걸 routing이라고 함.
챗 봇이 몇월 며칠에 있었던 정보들이 다 필요해서 slack, code, notion을 봐야한다면, 검색을 동시에 병렬론 실행을 해버림.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')


In [ ]:
from typing import Annotated, Literal, TypedDict
import operator

# TypedDict -> "딕셔너리"인데, key의 이름과 value의 데이터 형태를 미리 사전 정의하는 용도. 

# 하위 에이전트에 전달되는 간단한 state (query) -> dict
# {'query': '~~~~~'}
class AgentInput(TypedDict):
    """하위 Agent를 위한 input 형태"""
    query: str

# 하위 에이전트가 생성한 결과(source, result)
# {'source': 'slack', 'result': 'asdfsd'}
class AgentOutput(TypedDict):
    """하위 Agent들이 내놓는 output 형태"""
    source: str
    result: str

# 분류
# {'source': 'slack', 'query': '32sdfas'}
class Classification(TypedDict):
    """라우팅 결정: 어떤 agent에게 어떤 query를 넘길지"""
    source: Literal['github', 'notion', 'slack']
    query: str


# 주요 워크플로우 상태를 추적
class RouterState(TypedDict):
    # 사용자 질문
    query: str
    classifications: list[Classification]
    # Annotated: 주석을 다는 것.
    results: Annotated[list[AgentOutput], operator.add] 
    # 최종답변
    final_answer: str

# operator.add는 자동으로 더하는 기능
# 처음엔 비어있는 리스트였는데, 시간이 지나면 그 뒤에 자동으로 하나씩 더해라.
# [] + [{'source': 'github', 'result': 'asdfsd'}] + [{'source': 'slack', 'result': 'asdf2d'}]

In [ ]:
AgentInput(query='하이')
AgentOutput(source='github', result='good')

위 RouterState가 최종적으로 만들어내는 데이터 형태

```py
{
    'query': '~~~',
    'classification': [
        {'source': 'slack', 'query': '32sdfas'},
        {'source': 'github', 'query': '32sdfas'},
        {'source': 'notion', 'query': '32sdfas'},
    ],
    'results': [
        {'source': 'github', 'result': 'asdfsd'},
        {'source': 'slack', 'result': 'asdf2d'},
        {'source': 'notion', 'result': '32sdfas'},
    ],
    'final_answer': '그래서 ~~합니다.'
}
```

In [ ]:
# tool 만들기
from langchain.tools import tool


@tool
def search_code(query: str, repo: str = "main") -> str:
    """Search code in GitHub repositories."""
    return f"Found code matching '{query}' in {repo}: authentication middleware in src/auth.py"


@tool
def search_issues(query: str) -> str:
    """Search GitHub issues and pull requests."""
    return f"Found 3 issues matching '{query}': #142 (API auth docs), #89 (OAuth flow), #203 (token refresh)"


@tool
def search_prs(query: str) -> str:
    """Search pull requests for implementation details."""
    return f"PR #156 added JWT authentication, PR #178 updated OAuth scopes"


@tool
def search_notion(query: str) -> str:
    """Search Notion workspace for documentation."""
    return f"Found documentation: 'API Authentication Guide' - covers OAuth2 flow, API keys, and JWT tokens"


@tool
def get_page(page_id: str) -> str:
    """Get a specific Notion page by ID."""
    return f"Page content: Step-by-step authentication setup instructions"


@tool
def search_slack(query: str) -> str:
    """Search Slack messages and threads."""
    return f"Found discussion in #engineering: 'Use Bearer tokens for API auth, see docs for refresh flow'"


@tool
def get_thread(thread_id: str) -> str:
    """Get a specific Slack thread."""
    return f"Thread discusses best practices for API key rotation"

In [ ]:
# 각 전문 하위 에이전트 만들기

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4.1")

github_agent = create_agent(
    model,
    tools=[search_code, search_issues, search_prs],
    system_prompt=(
        "You are a GitHub expert. Answer questions about code, "
        "API references, and implementation details by searching "
        "repositories, issues, and pull requests."
    ),
)

notion_agent = create_agent(
    model,
    tools=[search_notion, get_page],
    system_prompt=(
        "You are a Notion expert. Answer questions about internal "
        "processes, policies, and team documentation by searching "
        "the organization's Notion workspace."
    ),
)

slack_agent = create_agent(
    model,
    tools=[search_slack, get_thread],
    system_prompt=(
        "You are a Slack expert. Answer questions by searching "
        "relevant threads and discussions where team members have "
        "shared knowledge and solutions."
    ),
)

In [ ]:
# 라우팅(어떤 하위 agent를 호출해야하는지 결정하는) LLM 만들기
router_model = init_chat_model('gpt-4.1-mini')

# StructuredOutput을 통해서, 정해진 포맷으로 LLM 결과 받는 Node 만들기
# pydantic은 데이터 형태를 설명하기 위한 것. 
from pydantic import BaseModel, Field

class RouterResult(BaseModel):
    """사용자 질문을 어떤 하위 에이전트에게 보낼지 결정한 결과를 만드는 용도"""
    classifications: list[Classification] = Field(
        description='실행할 목표 에이전트들과, 물어볼 질문들 목록'
    )


def route_query(state: RouterState) -> dict:
    """사용자 질문을 분류하고, 어떤 에이전트를 호출할지 결정"""
    s_llm = router_model.with_structured_output(RouterResult)
    

    result = s_llm.invoke([
        {
            "role": "system",
            "content": """Analyze this query and determine which knowledge bases to consult.
For each relevant source, generate a targeted sub-question optimized for that source.

Available sources:
- github: Code, API references, implementation details, issues, pull requests
- notion: Internal documentation, processes, policies, team wikis
- slack: Team discussions, informal knowledge sharing, recent conversations

Return ONLY the sources that are relevant to the query. Each source should have
a targeted sub-question optimized for that specific knowledge domain.

Example for "How do I authenticate API requests?":
- github: "What authentication code exists? Search for auth middleware, JWT handling"
- notion: "What authentication documentation exists? Look for API auth guides"
(slack omitted because it's not relevant for this technical question)"""
        },
        {"role": "user", "content": state["query"]}
    ])

    return {"classifications": result.classifications}

In [ ]:
# 분류함
route_query({'query': 'API 이니증은 어케함?'})

In [ ]:
from langgraph.types import Send

# 분류된 에이전트를 실제 호출
def route_to_agents(state: RouterState) -> list[Send]:
    """결정된 하위 agent 병렬 호출"""
    result = []
    for c in state['classifications']:
        result.append(Send(c['source'], {'query': c['query']}))
    return result

# notion, question A를 Send, github question B를 Send
# route_to_agents(
# {'classifications': 
#     [{'source': 'notion',
#     'query': 'question A'},
#     {'source': 'github',
#     'query': 'question B'}]}
# )



# github agent한테 일을 시키는 함수
def query_github(state: AgentInput) -> dict:
    """Query the GitHub agent."""
    result = github_agent.invoke({
        "messages": [{"role": "user", "content": state["query"]}]  
    })
    return {"results": [{"source": "github", "result": result["messages"][-1].content}]}


# notion agent한테 일을 시키는 함수
def query_notion(state: AgentInput) -> dict:
    """Query the Notion agent."""
    result = notion_agent.invoke({
        "messages": [{"role": "user", "content": state["query"]}]  
    })
    return {"results": [{"source": "notion", "result": result["messages"][-1].content}]}

# slack agent한테 일을 시키는 함수
def query_slack(state: AgentInput) -> dict:
    """Query the Slack agent."""
    result = slack_agent.invoke({
        "messages": [{"role": "user", "content": state["query"]}]  
    })
    return {"results": [{"source": "slack", "result": result["messages"][-1].content}]}

# 최종 답변을 종합하는 함수
def synthesize_results(state: RouterState) -> dict:
    """Combine results from all agents into a coherent answer."""
    if not state["results"]:
        return {"final_answer": "No results found from any knowledge source."}

    # Format results for synthesis
    formatted = [
        f"**From {r['source'].title()}:**\n{r['result']}"
        for r in state["results"]
    ]

    synthesis_response = router_model.invoke([
        {
            "role": "system",
            "content": f"""Synthesize these search results to answer the original question: "{state['query']}"

- Combine information from multiple sources without redundancy
- Highlight the most relevant and actionable information
- Note any discrepancies between sources
- Keep the response concise and well-organized"""
        },
        {"role": "user", "content": "\n\n".join(formatted)}
    ])

    return {"final_answer": synthesis_response.content}

In [ ]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(RouterState)

from langgraph.graph import StateGraph, START, END

workflow = (
    StateGraph(RouterState)
    .add_node("route", route_query)
    .add_node("github", query_github)
    .add_node("notion", query_notion)
    .add_node("slack", query_slack)
    .add_node("synthesize", synthesize_results)
    .add_edge(START, "route")
    .add_conditional_edges("route", route_to_agents, ["github", "notion", "slack"])
    .add_edge("github", "synthesize")
    .add_edge("notion", "synthesize")
    .add_edge("slack", "synthesize")
    .add_edge("synthesize", END)
    .compile()
)

from IPython.display import display, Image

display(Image(workflow.get_graph().draw_mermaid_png()))

In [ ]:
result = workflow.invoke({
    "query": "How do I authenticate API requests?"
})

print("Original query:", result["query"])
print("\nClassifications:")
for c in result["classifications"]:
    print(f"  {c['source']}: {c['query']}")
print("\n" + "=" * 60 + "\n")
print("Final Answer:")
print(result["final_answer"])